![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]
qb = QuantBook()
self = qb
msft = self.AddEquity("MSFT")

market = self.AddEquity("SPY")

history = self.History(self.Securities.Keys, 365, Resolution.Daily)

#Target Values
rocp = self.Indicator(RateOfChangePercent(1), msft.Symbol, 365, Resolution.Daily)
rocp.plot()

# Indicator Analysis
rsi = self.Indicator(RelativeStrengthIndex(1), msft.Symbol, 365, Resolution.Daily)
rsi = rsi.drop('averagegain', 1)
rsi = rsi.drop('averageloss', 1)
print(rsi.plot())

market_rocp = self.Indicator(RateOfChangePercent(1), market.Symbol, 365, Resolution.Daily)
market_rocp.plot()

atr = self.Indicator(AverageTrueRange(1), msft.Symbol, 364, Resolution.Daily)
atr.drop('truerange', 1).plot()


In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV

import sklearn as sk
import xgboost as xgb
import pandas as pd
import numpy as np

scaler = MinMaxScaler(feature_range=(-1, 1))

scaler.fit(rocp)
scaled_rocp = scaler.transform(rocp)

scaler.fit(market_rocp)
scaled_market_rocp = scaler.transform(market_rocp)
scaler.fit(rsi)
scaled_rsi = scaler.transform(rsi)
scaler.fit(atr)
scaled_atr = scaler.transform(atr)

# create our indicator dataframe to feed into our train_test_split funtion
indicator_df = pd.DataFrame(np.hstack((scaled_market_rocp, scaled_rsi, scaled_atr)))
print(indicator_df)




In [ ]:
# split our data into training and testing groups for our model
x_train, x_valid, y_train, y_valid = train_test_split(indicator_df, rocp, test_size=0.35, random_state=42)

# fit the model to the data with the eval set with the parameters below passed to the random search optimizer with 5 iterations and a cv of 4
parameters = {
        'n_estimators': [100, 200, 300, 400],
        'learning_rate': [0.001, 0.005, 0.01, 0.05],
        'max_depth': [8, 10, 12, 15],
        'gamma': [0.001, 0.005, 0.01, 0.02],
        'random_state': [42]
     }
eval_set = [(x_train, y_train), (x_valid, y_valid)]
model = xgb.XGBRegressor(eval_set=eval_set, objective='reg:squarederror', verbose=False)

model = RandomizedSearchCV(estimator=model, param_distributions=parameters, n_iter=5, scoring='neg_mean_squared_error', cv=4, verbose=1)

model.fit(x_train, y_train, eval_set=eval_set, verbose=False)

score = model.score
print(score)